In [1]:
import numpy as np
import pandas as pd

Loading the data into a pandas df

In [4]:
pop_df = pd.read_csv("Population_by_State__County_and_City__2010_to_Present.csv")

In [6]:
ev_df = pd.read_csv("Electric_Vehicle_Population_Data.csv")

In [8]:
ev_pop = pd.read_csv("Electric_Vehicle_Population_Size_History_By_County.csv")

In [10]:
#removing unwanted columns
pop_df = pop_df.drop(['POP_2011', 'POP_2012', 'POP_2013', 'POP_2014', 'POP_2015', 'POP_2016'], axis = 1)

In [12]:
pop_df = pop_df[pop_df['FILTER'].isin([2, 3])]

In [14]:
pop_df.head()

,SEQUENCE,FILTER,COUNTY,JURISDICTION,POP_2017,POP_2018,POP_2019,POP_2020,POP_2021,POP_2022,POP_2023,POP_2024
1,2,2,Adams,Unincorporated Adams County,9250,9301,9375,9472,9575,9575,9585,9628
2,3,3,Adams,Incorporated Adams County,10818,10899,10960,11141,11325,11525,11615,11847
9,10,2,Asotin,Unincorporated Asotin County,13627,13734,13816,13920,14070,14165,14205,14260
10,11,3,Asotin,Incorporated Asotin County,8420,8378,8366,8365,8430,8435,8445,8465
14,15,2,Benton,Unincorporated Benton County,35250,35887,36197,36556,36760,37445,37615,37805


In [16]:
ev_df = ev_df.drop(['Legislative District', 'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility', '2020 Census Tract'], axis = 1)

In [18]:
# renaming columns for SQL
ev_df = ev_df.rename(columns={'Postal Code': 'zip', 'Model Year': 'Model_Year', 'Electric Vehicle Type': 'EV_type',
                          'Clean Alternative Fuel Vehicle (CAFV) Eligibility': 'CAFV_eligibility', 'Electric Range': 'Range', 
                          'Base MSRP': 'MSRP', 'VIN (1-10)': 'VIN'})

In [20]:
ev_df.head()

,VIN,County,City,State,zip,Model_Year,Make,Model,EV_type,CAFV_eligibility,Range,MSRP
0,5UXTA6C0XM,Kitsap,Seabeck,WA,98380.0,2021,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30.0,0.0
1,5YJ3E1EB1J,Kitsap,Poulsbo,WA,98370.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215.0,0.0
2,WP0AD2A73G,Snohomish,Bothell,WA,98012.0,2016,PORSCHE,PANAMERA,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,15.0,0.0
3,5YJ3E1EB5J,Kitsap,Bremerton,WA,98310.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215.0,0.0
4,1N4AZ1CP3K,King,Redmond,WA,98052.0,2019,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,150.0,0.0


In [22]:
ev_pop = ev_pop[ev_pop['State'] == 'WA']

In [24]:
ev_pop.loc[:, 'Date'] = pd.to_datetime(ev_pop['Date'], format='%B %d %Y')

In [26]:
#re-indexing df4
ev_pop.index = range(0, len(ev_pop))

In [28]:
ev_pop = ev_pop.rename(columns={'Plug-In Hybrid Electric Vehicles (PHEVs)': 'phev', 'Battery Electric Vehicles (BEVs)': 'bev',
                               'Vehicle Primary Use': 'primary_use', 'Electric Vehicle (EV) Total': 'ev', 'Total Vehicles': 'total_vehicles',
                               'Percent Electric Vehicles': 'percent_ev', 'Non-Electric Vehicle Total': 'Non_ev'})

In [30]:
ev_pop.head()

,Date,County,State,primary_use,bev,phev,ev,Non_ev,total_vehicles,percent_ev
0,2017-12-31 00:00:00,Pend Oreille,WA,Truck,0,0,0,5619,5619,0.00
1,2017-04-30 00:00:00,Lincoln,WA,Truck,0,0,0,4464,4464,0.00
2,2019-03-31 00:00:00,San Juan,WA,Passenger,197,70,267,14284,14551,1.83
3,2021-03-31 00:00:00,Asotin,WA,Truck,0,0,0,6721,6721,0.00
4,2024-04-30 00:00:00,Clallam,WA,Truck,30,0,30,21408,21438,0.14


loading the data onto the staging area

In [ ]:
from sqlalchemy import create_engine
import urllib


server = '***************'
database = '*************'

connection_string = urllib.parse.quote_plus(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};DATABASE={database};'
    'Trusted_Connection=yes;'
)

engine = create_engine(f'mssql+pyodbc:///?odbc_connect={connection_string}')

pop_df.to_sql('pop_df', con=engine, if_exists='append', index=False)
ev_df.to_sql('ev_df', con=engine, if_exists='append', index=False)
ev_pop.to_sql('ev_pop', con=engine, if_exists='append', index=False)